# Brewery Tutorial
__[Open Brewery](https://www.openbrewerydb.org)__ DB is a free dataset and API with public information on breweries, cideries, brewpubs, and bottleshops. The goal of Open Brewery DB is to maintain an open-source, community-driven dataset and provide a public API. Datasets provided by the project are available in the following formats:
- __[CSV](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.csv)__
- __[JSON](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.json)__
- __[PostgreSQL SQL](https://github.com/openbrewerydb/openbrewerydb/blob/master/breweries.sql)__

For this tutorial, CSV will be used.

TerminusDB Server must be installed on your system before running the Python script. Follow the instructions on __[terminusdb-bootstrap](https://github.com/terminusdb/terminusdb-bootstrap)__. terminusdb-server will be running as a Docker container on http://127.0.0.1:6363.

## Scaffolding
Database and schema creation in TerminusDB can be managed through the scaffolding tool available from the command line.

- Create a new project

In [ ]:
terminusdb startproject

This command will create a schema template (̣̣`schema.py`) and configuration file (`settings.py`) into current working directory. `settings.py` must be customized with the name of the database for the project. `schema.py`  
must be edited and schema definition replaced according to the data imported from the CSV file.

- Update schema

In [ ]:
terminusdb commit

Database will be created if not exists and connection established. Schema will be updated in TerminusDB.

- Sync schema

In [ ]:
terminusdb sync

`schema.py` will be updated with database schema

- Delete database

In [ ]:
terminusdb deletedb database_name

## Schema creation
The dataset has the following columns:
- obdb_id
- name
- brewery_type
- street
- address_2
- address_3
- city
- state
- county_province
- postal_code
- website_url
- phone
- created_at
- updated_at
- country
- longitude
- latitude
- tags

Some of which are optional and rarely have a value assigned and can be omitted when creating the schema and importing the values.

Analyzing the dataset:

- A brewery has *name*, *type*, *address*, *phone* and *website url*
- A brewery can be any of ten different types
- An address is a group of values that include *street*, *city*, *postal code* and *coordinates*
- A city is located in a state
- A state is part of a country
- Coordinates are a pair of values, longitude and latitude

Based on what's described above, the following documents are created, each class represents a document in the schema:
- Brewery
- Brewrey_Type
- Address
- City
- State
- Country
- Coordinates

IDs are created using `ValueHashKey` or `RandomKey`.

`schema.py` must be customized according to the documents required and values imported from the CSV file.

Don't forget to update and sync the schema once changes are made.

In [ ]:
from typing import List

from terminusdb_client.woqlschema import DocumentTemplate, RandomKey, ValueHashKey

class Address(DocumentTemplate):
    _subdocument = []
    city: "City"
    coordinates: List["Coordinates"]
    postal_code: str
    street: str


class Brewery(DocumentTemplate):
    _key = RandomKey()
    address_of: "Address"
    name: str
    phone: str
    type_of: "Brewery_Type"
    website_url: str


class Brewery_Type(DocumentTemplate):
    _key = ValueHashKey()
    name: str


class City(DocumentTemplate):
    _key = ValueHashKey()
    name: str
    state: "State"


class Coordinates(DocumentTemplate):
    _key = RandomKey()
    latitude: float
    longitude: float


class Country(DocumentTemplate):
    _key = ValueHashKey()
    name: str


class State(DocumentTemplate):
    _key = ValueHashKey()
    country: "Country"
    name: str

## Import libraries
A Python script must be created for transfering data from a CSV file to a TerminusDB database. Python client of TerminusDB and pandas are required. To import these libraries in the script, add the following lines:

In [ ]:
from typing import List
from terminusdb_client import WOQLQuery, WOQLClient
import pandas as pd

## Database management
Using the Python client:
- Establish a connection to TerminusDB
- Import data from CSV file
- Print data from TerminusDB

In [ ]:
if __name__ == "__main__":
    db_id = "open_brewery"
    url = "https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv"
    client = WOQLClient("http://127.0.0.1:6363")
    client.connect(key="root", account="admin", user="admin")
    client.set_db(db_id)
    insert_data(client, url)
    results = client.get_all_documents(graph_type="instance", count=2)
    print(list(results))

## Transfer data
pandas provides built-in functions that make it simple to read and extract data from a CSV file. `read_csv` receives the path of the file as parameter, it can be a URL or a local file. Columns can be specified if not all are required, with `usecols`.

As some cells in the CSV don't have a value, when importing data it would be required to replace NULL values with '' if an integer or float is expected instead. `fillna` function must be called to avoid 'Not a number' errors.

In [ ]:
def insert_data(client, url):
    all_breweries = []
    df = pd.read_csv(url, usecols = ['name', 'brewery_type', 'street', 'city', 'state', 'postal_code', 'website_url','phone', 'country', 'longitude', 'latitude'])
    df = df.fillna('')

Using a for loop, iterate through the values in the CSV, create objects for each document in the schema, assign values to the corresponding variables, and append these values to the `all_breweries` list.

In [ ]:
    for index, row in df.iterrows():
        country = Country()
        country.name = row['country']
        state = State()
        state.name = row['state']
        state.country = country
        city = City()
        city.name = row['city']
        city.state = state
        address = Address()
        address.street = row['street']
        address.city = city
        address.postal_code = row['postal_code']
        address.coordinates = [str(row['longitude']), str(row['latitude'])]
        brewery_type = Brewery_Type()
        brewery_type.name = row['brewery_type']
        brewery = Brewery()
        brewery.type_of = brewery_type
        brewery.address_of = address
        brewery.phone = row['phone']
        brewery.website_url = row['website_url']
        all_breweries.append(brewery)

Insert the `all_breweries` list into TerminusDB, with ``insert_document`.

In [ ]:
    client.insert_document(all_breweries,
                           commit_msg="Adding all breweries")